In [ ]:
# Numpy and pandas by default assume a narrow screen - this fixes that

from fastai.vision.all import *

from ipywidgets import widgets
from pandas.api.types import CategoricalDtype
import os
import matplotlib as mpl
# mpl.rcParams['figure.dpi']= 200
mpl.rcParams['savefig.dpi']= 200
mpl.rcParams['font.size']=12

set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
pd.set_option('display.max_columns',999)
np.set_printoptions(linewidth=200)
torch.set_printoptions(linewidth=200)
import pandas as pd


In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
print(len(df))
print(df.columns)
# print(train['labels'].value_counts())
print(df['labels'].value_counts().plot.bar())

In [ ]:
im = Image.open('../input/plant-pathology-2021-fgvc8/train_images/800113bb65efe69e.jpg')
im.thumbnail((256,256))
im

In [ ]:
import pandas as pd
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')


path = Path('../input/plant-pathology-2021-fgvc8')
Path.BASE_PATH=path
path.ls()
df.shape

In [ ]:
count_dict = df.labels.value_counts()
count_dict

In [ ]:
def get_x(r): return path/'train_images'/r['image']
def get_y(r): return r['labels'].split(' ')
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   get_x = get_x, get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[2]

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
               
                   get_x=get_x, 
                   get_y=get_y,
                   item_tfms = RandomResizedCrop(244, min_scale=0.55))
dls = dblock.dataloaders(df)
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.37), model_dir="/kaggle/working")


In [ ]:
learn.fine_tune(2, base_lr=3e-3, freeze_epochs=2)

In [ ]:
preds,targs = learn.get_preds()
xs = torch.linspace(0.05,0.95,29)
accs = [accuracy_multi(preds, targs, thresh=i, sigmoid=False) for i in xs]
plt.plot(xs,accs);

In [ ]:
learn.lr_find()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)


In [ ]:
learn.save("model")

In [ ]:
interp.most_confused(min_val=5)

In [ ]:
files=get_image_files(path/'test_images')
learn.predict(files[1])

In [ ]:
im = Image.open('../input/plant-pathology-2021-fgvc8/test_images/ad8770db05586b59.jpg')
im.thumbnail((256,256))
im

In [ ]:

im = Image.open('../input/plant-pathology-2021-fgvc8/test_images/c7b03e718489f3ca.jpg')
im.thumbnail((256,256))
im

In [ ]:
files=get_image_files(path/'test_images')
learn.predict(files[0])


In [ ]:
im = Image.open('../input/plant-pathology-2021-fgvc8/test_images/85f8cb619c66b863.jpg')
im.thumbnail((256,256))
im

In [ ]:
files=get_image_files(path/'test_images')
learn.predict(files[2])


In [ ]:
test_dl = dls.test_dl(get_image_files(path/'test_images'))
dl=learn.get_preds(dl=test_dl)
dl

In [ ]:
sample_submission = pd.DataFrame({
"image":['85f8cb619c66b863.jpg','ad8770db05586b59.jpg','c7b03e718489f3ca.jpg'],
"labels":['complex and frog_eye_leaf_spot','frog_eye_leaf_spot','no label']})
submission.to_csv('submission.csv', index=False)

In [ ]:
sample_submission=pd.read_csv('./submission.csv')
sample_submission